## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


In [2]:
!cd /home/dsc/Data/challenge

In [3]:
!pwd

/home/dsc/Data/challenge


In [4]:
!ls -l

total 1014320
-rw-rw-r--. 1 dsc dsc 554970628 Jan  4  2016 bookings.csv.bz2
-rw-rw-r--  1 dsc dsc     54834 May  9 22:50 bookings.sample.csv.bz2
-rw-rw-r--  1 dsc dsc    425006 May  9 22:51 bookings.sample.csv.csv
-rw-rw-r--. 1 dsc dsc 483188920 Jan  4  2016 searches.csv.bz2


## 1) Get familiar with data

In [5]:
import pandas as pd

In [22]:
#DON'T DO THIS!!!
b = pd.read_csv('bookings.sample.csv.bz2',sep='^')

### What if we dont want to read the whole file?

In [6]:
b=pd.read_csv('./bookings.sample.csv.bz2', sep='^')

In [7]:
#read just the first nrows from the file
b=pd.read_csv('./bookings.csv.bz2', sep='^', nrows=1000)

In [8]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [9]:
#by default we don't see all the colums so we remove the contraints on displaying the number of columns
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_columns',5) #with this we would see only 5 columns when using head/tail/sample

In [10]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [11]:
b.describe()

,duration,distance,intl,pax,year,month
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.0
mean,23935.433000,407.569000,0.643000,0.532000,2013.0,3.0
std,54459.101785,1490.087967,0.479355,1.976077,0.0,0.0
min,61.000000,0.000000,0.000000,-8.000000,2013.0,3.0
25%,3493.000000,0.000000,0.000000,-1.000000,2013.0,3.0
50%,6891.000000,0.000000,1.000000,1.000000,2013.0,3.0
75%,17940.000000,0.000000,1.000000,1.000000,2013.0,3.0
max,482174.000000,10995.000000,1.000000,21.000000,2013.0,3.0


In [12]:
b.dtypes

act_date               object
source                 object
pos_ctry               object
pos_iata               object
pos_oid                object
rloc                   object
cre_date               object
duration                int64
distance                int64
dep_port               object
dep_city               object
dep_ctry               object
arr_port               object
arr_city               object
arr_ctry               object
lst_port               object
lst_city               object
lst_ctry               object
brd_port               object
brd_city               object
brd_ctry               object
off_port               object
off_city               object
off_ctry               object
mkt_port               object
mkt_city               object
mkt_ctry               object
intl                    int64
route                  object
carrier                object
bkg_class              object
cab_class              object
brd_time               object
off_time  

In [13]:
b.isnull().sum()

act_date               0
source                 0
pos_ctry               0
pos_iata               0
pos_oid                0
rloc                   0
cre_date               0
duration               0
distance               0
dep_port               0
dep_city               0
dep_ctry               0
arr_port               0
arr_city               0
arr_ctry               0
lst_port               0
lst_city               0
lst_ctry               0
brd_port               0
brd_city               0
brd_ctry               0
off_port               0
off_city               0
off_ctry               0
mkt_port               0
mkt_city               0
mkt_ctry               0
intl                   0
route                  0
carrier                0
bkg_class              0
cab_class              0
brd_time               0
off_time               0
pax                    0
year                   0
month                  0
oid                    0
dtype: int64

In [14]:
b.count()

act_date               1000
source                 1000
pos_ctry               1000
pos_iata               1000
pos_oid                1000
rloc                   1000
cre_date               1000
duration               1000
distance               1000
dep_port               1000
dep_city               1000
dep_ctry               1000
arr_port               1000
arr_city               1000
arr_ctry               1000
lst_port               1000
lst_city               1000
lst_ctry               1000
brd_port               1000
brd_city               1000
brd_ctry               1000
off_port               1000
off_city               1000
off_ctry               1000
mkt_port               1000
mkt_city               1000
mkt_ctry               1000
intl                   1000
route                  1000
carrier                1000
bkg_class              1000
cab_class              1000
brd_time               1000
off_time               1000
pax                    1000
year                

In [15]:
b.tail()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
995,2013-03-23 00:00:00,1P,US,bfafedb1505531ad8e70e176941a7ad0,804deeaddf16e6a93d36e5d046040d60,be83fd657625d53f58c5d3070cdb41b8,2013-03-06 00:00:00,5570,0,SLC,SLC,US,MSN,MSN,US,SLC,SLC,US,SLC,SLC,US,MSN,MSN,US,MSNSLC,MSNSLC,USUS,0,SLCMSN,NV,K,Y,2013-05-20 14:00:00,2013-05-20 18:00:55,1,2013,3,NULL
996,2013-03-06 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,691f15622668e671f51ba033ea904398,2013-03-06 00:00:00,4664,0,SNA,SNA,US,SFO,SFO,US,SNA,SNA,US,SNA,SNA,US,SFO,SFO,US,SFOSNA,SFOSNA,USUS,0,SNASFO,FK,V,Y,2013-03-17 13:31:00,2013-03-17 15:01:12,1,2013,3,NULL
997,2013-03-06 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,691f15622668e671f51ba033ea904398,2013-03-06 00:00:00,4664,0,SNA,SNA,US,SFO,SFO,US,SNA,SNA,US,SFO,SFO,US,SNA,SNA,US,SFOSNA,SFOSNA,USUS,0,SFOSNA,FK,W,Y,2013-03-20 17:45:00,2013-03-20 19:15:12,1,2013,3,NULL
998,2013-03-06 00:00:00,1P,US,0deb0937d0f143f79505297b235eaeeb,f4ae494f0ebec1d84f6e01750d61747d,bed521f09feee94fcb87f705267b0327,2013-03-06 00:00:00,331,2598,LAX,LAX,US,MEM,MEM,US,MEM,MEM,US,LAX,LAX,US,MEM,MEM,US,LAXMEM,LAXMEM,USUS,0,LAXDFWMEM,KK,Y,Y,2013-03-09 07:30:00,2013-03-09 15:01:52,1,2013,3,NULL
999,2013-03-07 00:00:00,1P,US,0deb0937d0f143f79505297b235eaeeb,f4ae494f0ebec1d84f6e01750d61747d,bed521f09feee94fcb87f705267b0327,2013-03-06 00:00:00,331,2598,LAX,LAX,US,MEM,MEM,US,MEM,MEM,US,LAX,LAX,US,MEM,MEM,US,LAXMEM,LAXMEM,USUS,0,LAXDFWMEM,KK,Y,Y,2013-03-09 07:30:00,2013-03-09 15:01:52,-1,2013,3,NULL


In [16]:
b.columns

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')

In [17]:
b.columns.tolist()

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [18]:
b.columns=b.columns.str.strip()

In [19]:
b.columns

Index([u'act_date', u'source', u'pos_ctry', u'pos_iata', u'pos_oid', u'rloc',
       u'cre_date', u'duration', u'distance', u'dep_port', u'dep_city',
       u'dep_ctry', u'arr_port', u'arr_city', u'arr_ctry', u'lst_port',
       u'lst_city', u'lst_ctry', u'brd_port', u'brd_city', u'brd_ctry',
       u'off_port', u'off_city', u'off_ctry', u'mkt_port', u'mkt_city',
       u'mkt_ctry', u'intl', u'route', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time', u'off_time', u'pax', u'year', u'month', u'oid'],
      dtype='object')

## 2) Select the columns of interest 

In [20]:
b=pd.read_csv('bookings.sample.csv.bz2', sep='^', usecols=['arr_port','pax','year'])

In [21]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 3) What to do with NaN?



In [22]:
b.isnull().sum()

arr_port    0
pax         0
year        0
dtype: int64

In [23]:
b=b.dropna()

#### What does df.dropna() do?

In [24]:
import numpy as np
import pandas as pd
data=np.random.randn(5,2)
df=pd.DataFrame(data, columns=['one','two'])
df

,one,two
0,1.016081,0.272905
1,-1.133663,0.538976
2,0.427253,-2.180686
3,0.788693,-1.097824
4,-0.162133,-1.669835


In [25]:
df['three']=[None, None, 1, 2, None]

In [26]:
df

,one,two,three
0,1.016081,0.272905,NaN
1,-1.133663,0.538976,NaN
2,0.427253,-2.180686,1.0
3,0.788693,-1.097824,2.0
4,-0.162133,-1.669835,NaN


In [27]:
df.dropna()

,one,two,three
2,0.427253,-2.180686,1.0
3,0.788693,-1.097824,2.0


In [28]:
df.dropna(how='all')

,one,two,three
0,1.016081,0.272905,NaN
1,-1.133663,0.538976,NaN
2,0.427253,-2.180686,1.0
3,0.788693,-1.097824,2.0
4,-0.162133,-1.669835,NaN


In [29]:
df.dropna(how='any', axis=1)

,one,two
0,1.016081,0.272905
1,-1.133663,0.538976
2,0.427253,-2.180686
3,0.788693,-1.097824
4,-0.162133,-1.669835


#### What if we want to drop just based on second column NaN?

In [30]:
df['three']=[None, None, 1, 2, None]
df['two'][1]=None
df['two'].isnull()

0    False
1     True
2    False
3    False
4    False
Name: two, dtype: bool

In [31]:
~df['two'].isnull()

0     True
1    False
2     True
3     True
4     True
Name: two, dtype: bool

In [32]:
df[~(df['two'].isnull())]

,one,two,three
0,1.016081,0.272905,NaN
2,0.427253,-2.180686,1.0
3,0.788693,-1.097824,2.0
4,-0.162133,-1.669835,NaN


### What do we have so far?

In [33]:
import pandas as pd
b=pd.read_csv('bookings.sample.csv.bz2', sep='^', usecols=['arr_port','pax','year'])
b=b.dropna()
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 4) make action plan


In [34]:
b2 = b[b['year']==2013]
print len(b)
print len(b2)

999
999


In [35]:
b = b[b['year']==2013]
b.drop('year',axis=1, inplace=True)
b.head()

,arr_port,pax
0,LHR,-1
1,CLT,1
2,CLT,1
3,SVO,1
4,SVO,1


In [36]:
arr_ports=b.groupby('arr_port')
type(arr_ports)
arr_ports

In [37]:
for k, (airport, val) in enumerate(arr_ports):
    print '%s' %(airport)
    print val['pax'].sum()
    if k==1:
        break

ACC     
6
ADD     
2


In [38]:
arr_ports.get_group('ACC     ')

,arr_port,pax
118,ACC,3
119,ACC,3


In [39]:
arr_ports.groups

{'ACC     ': Int64Index([118, 119], dtype='int64'),
 'ADD     ': Int64Index([207, 208, 301, 302], dtype='int64'),
 'AGP     ': Int64Index([397, 398], dtype='int64'),
 'ALA     ': Int64Index([76, 77], dtype='int64'),
 'ALC     ': Int64Index([909, 910], dtype='int64'),
 'ALG     ': Int64Index([34, 35, 36, 37, 558, 559], dtype='int64'),
 'AMD     ': Int64Index([732], dtype='int64'),
 'AMS     ': Int64Index([494, 888, 889, 890, 891], dtype='int64'),
 'ANC     ': Int64Index([432, 433], dtype='int64'),
 'ASU     ': Int64Index([548, 549, 550, 551, 724, 725], dtype='int64'),
 'ATH     ': Int64Index([313, 314, 739, 748], dtype='int64'),
 'ATL     ': Int64Index([147, 404, 908], dtype='int64'),
 'ATQ     ': Int64Index([801, 802, 803], dtype='int64'),
 'AUH     ': Int64Index([733, 734], dtype='int64'),
 'AUS     ': Int64Index([380, 381], dtype='int64'),
 'AYT     ': Int64Index([81, 82, 83, 500], dtype='int64'),
 'BBI     ': Int64Index([151, 152], dtype='int64'),
 'BCN     ': Int64Index([320, 321, 

In [40]:
arr_ports=b.groupby('arr_port')
arr_ports.sum().head()
type(arr_ports.sum().head())

pandas.core.frame.DataFrame

In [41]:
arr_ports.sum().sort_values('pax', ascending=False).head(10)

,pax
arr_port,
DCA,50
JFK,14
FLL,13
PVG,13
CGK,12
LGA,12
SVO,12
LAS,11
BOS,11


## 5) Adjust the code to work with Big data

In [42]:
import pandas as pd
b=pd.read_csv('bookings.sample.csv.bz2', sep='^', usecols=['arr_port','pax','year'])
b=b.dropna()
b = b[b['year']==2013]
b.drop('year',axis=1, inplace=True)
b.groupby('arr_port').sum().sort_values('pax', ascending=False).head(10)


,pax
arr_port,
DCA,50
JFK,14
FLL,13
PVG,13
CGK,12
LGA,12
SVO,12
LAS,11
BOS,11


#### use chunks to reat the whole file


#### How do we get the all the results of all chunks at one place?


#### 1) iterator=true and get_chunk

In [43]:
b=pd.read_csv('bookings.sample.csv.bz2', sep='^', \
              usecols=['arr_port','pax','year'])
print type(b)

b=pd.read_csv('bookings.sample.csv.bz2', sep='^', \
              usecols=['arr_port','pax','year'], iterator=True)
print type(b)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.io.parsers.TextFileReader'>


In [44]:
b0=b.get_chunk(600)

In [45]:
len(b0)

600

In [46]:
b1=b.get_chunk(300)

In [47]:
len (b1)

300

In [48]:
b2=b.get_chunk(300)

In [49]:
len(b2)

99

#### 2) define chunk size

In [50]:
bc=pd.read_csv('bookings.sample.csv.bz2', sep='^', \
              usecols=['arr_port','pax','year'], chunksize=600)
print type(bc)

<class 'pandas.io.parsers.TextFileReader'>


In [51]:
for i, chunk in enumerate(bc):
    print "Chunk number: %d"%i
    print "Chunk size %d"%len(chunk)
    print chunk.head()
    

Chunk number: 0
Chunk size 600
   arr_port  pax  year
0  LHR        -1  2013
1  CLT         1  2013
2  CLT         1  2013
3  SVO         1  2013
4  SVO         1  2013
Chunk number: 1
Chunk size 399
     arr_port  pax  year
600  PHL         1  2013
601  SFO         1  2013
602  BOG         1  2013
603  BOG         1  2013
604  BOG        -1  2013


In [52]:
for data_frame_of_chunk in bc:
    print "Chunk size %d"%len(data_frame_of_chunk)
    print data_frame_of_chunk.head()
    print type(data_frame_of_chunk)

In [53]:
#make it work with the sample before running the code over the original file!!!
import pandas as pd
bc=pd.read_csv('bookings.sample.csv.bz2', \
              sep='^', usecols=['arr_port','pax','year'], chunksize=400)

all_chunk_results=pd.DataFrame()
for (i, chunk) in enumerate(bc):
    print "Chunk Num %d"%i
    chunk=chunk.dropna()
    chunk = chunk[chunk['year']==2013]
    chunk.drop('year',axis=1, inplace=True)
    chunk_result=chunk.groupby('arr_port').sum()
    #.sort_values('pax', ascending=False)
    all_chunk_results=all_chunk_results.append(chunk_result)

topirports=all_chunk_results.reset_index().groupby('arr_port').sum()\
.sort_values('pax', ascending=False).reset_index().head(10)
topirports

Chunk Num 0
Chunk Num 1
Chunk Num 2


,arr_port,pax
0,DCA,50
1,JFK,14
2,FLL,13
3,PVG,13
4,CGK,12
5,LGA,12
6,SVO,12
7,LAS,11
8,BOS,11
9,MIA,11


In [54]:
%%time
import pandas as pd
bc=pd.read_csv('bookings.csv.bz2', \
              sep='^', usecols=['arr_port','pax','year'], chunksize=1000000)

all_chunk_results=pd.DataFrame()
for (i, chunk) in enumerate(bc):
    print "Chunk Num %d"%i
    chunk=chunk.dropna()
    chunk = chunk[chunk['year']==2013]
    chunk.drop('year',axis=1, inplace=True)
    chunk_result=chunk.groupby('arr_port').sum()
    #.sort_values('pax', ascending=False)
    all_chunk_results=all_chunk_results.append(chunk_result)

topirports=all_chunk_results.groupby(all_chunk_results.index).sum()\
.sort_values('pax', ascending=False).reset_index()
topirports

Chunk Num 0
Chunk Num 1
Chunk Num 2
Chunk Num 3
Chunk Num 4
Chunk Num 5
Chunk Num 6
Chunk Num 7
Chunk Num 8
Chunk Num 9
Chunk Num 10
CPU times: user 3min 43s, sys: 2.63 s, total: 3min 46s
Wall time: 3min 52s


In [55]:
topirports.shape

(2274, 2)

In [56]:
topirports.head()

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0


In [58]:
#Save the results to file
topirports.to_csv('top_airports.csv', sep='^', index=False)

In [59]:
!ls -l

total 1014356
-rw-rw-r--. 1 dsc dsc 554970628 Jan  4  2016 bookings.csv.bz2
-rw-rw-r--  1 dsc dsc     54834 May  9 22:50 bookings.sample.csv.bz2
-rw-rw-r--  1 dsc dsc    425006 May  9 22:51 bookings.sample.csv.csv
-rw-rw-r--. 1 dsc dsc 483188920 Jan  4  2016 searches.csv.bz2
-rw-rw-r--  1 dsc dsc     33984 May  9 23:00 top_airports.csv


In [60]:
#test importing the data from the file... just in case:)
top=pd.read_csv('top_airports.csv', sep='^')

In [61]:
top.head()

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0


### Now we've got top airports

lets do the bonus solution

In [63]:
from GeoBases import GeoBase

In [64]:
geoDict = GeoBase(data='ori_por', verbose=False)
#geoDict = GeoBase(data='ori_por')

In [65]:
type(geoDict)

GeoBases.GeoBaseModule.GeoBase

In [66]:
geoDict.keys()[:10]

['MLC',
 'XMZ',
 'XQP@1',
 'JED@1',
 'MFI@1',
 'JGO@1',
 'YAL@1',
 'NDP',
 'GEG@1',
 'NDS']

In [67]:
geoDict.fields

['__key__',
 '__dup__',
 '__par__',
 '__lno__',
 'iata_code',
 'icao_code',
 'faa_code',
 'is_geonames',
 'geoname_id',
 'valid_id',
 'name',
 'asciiname',
 'lat',
 'lng',
 'fclass',
 'fcode',
 'page_rank',
 'date_from',
 'date_until',
 'comment',
 'country_code',
 'cc2',
 'country_name',
 'continent_name',
 'adm1_code',
 'adm1_name_utf',
 'adm1_name_ascii',
 'adm2_code',
 'adm2_name_utf',
 'adm2_name_ascii',
 'adm3_code',
 'adm4_code',
 'population',
 'elevation',
 'gtopo30',
 'timezone',
 'gmt_offset',
 'dst_offset',
 'raw_offset',
 'moddate',
 'city_code',
 'city_name_utf',
 'city_name_ascii',
 'tvl_por_list@raw',
 'tvl_por_list',
 'state_code',
 'location_type@raw',
 'location_type',
 'wiki_link',
 'alt_name_section@raw',
 'alt_name_section',
 '__gar__']

In [68]:
geoDict.get?

In [69]:
geoDict.get('LHR','city_name_utf')

'London'

In [70]:
geoDict.get('LHR')['name']

'London Heathrow Airport'

In [71]:
import pandas as pd 
top=pd.read_csv('top_airports.csv', sep='^')
top.arr_port[0]

'LHR     '

#### we need to remove white spaces

In [72]:
top['arr_port'].map(lambda x: geoDict.get(x.strip())['name'])[:10]

0                 London Heathrow Airport
1           Orlando International Airport
2       Los Angeles International Airport
3         Mc Carran International Airport
4    John F Kennedy International Airport
5               Paris - Charles-de-Gaulle
6                            Suvarnabhumi
7             Miami International Airport
8     San Francisco International Airport
9             Dubai International Airport
Name: arr_port, dtype: object

In [73]:
top['Airport Name']=top['arr_port'].map(lambda x: geoDict.get(x.strip())['name'])
top.head()

,arr_port,pax,Airport Name
0,LHR,88809.0,London Heathrow Airport
1,MCO,70930.0,Orlando International Airport
2,LAX,70530.0,Los Angeles International Airport
3,LAS,69630.0,Mc Carran International Airport
4,JFK,66270.0,John F Kennedy International Airport
